In [1]:
from datasets import load_dataset, load_from_disk

In [2]:
import json
from tqdm import tqdm

with open('test.jsonl', 'w') as fopen:
    with open('wikipedia-2023-10-01.jsonl') as fopen_l:
        for l in tqdm(fopen_l):
            l = json.loads(l)
            d = {
                'text': l
            }
            fopen.write(f'{json.dumps(d)}\n')

438316it [00:08, 50713.87it/s] 


In [3]:
raw_datasets = load_dataset(
    'json',
    data_files='test.jsonl',
    split = 'train',
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    'malaysia-ai/bpe-tokenizer',
)
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False

In [6]:
def tokenize_function(examples):
    output = tokenizer(examples['text'])
    output.pop('token_type_ids', None)
    return output

In [7]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    remove_columns=['text'],
    batched=True,
    num_proc = 4
)

Map (num_proc=4):   0%|          | 0/438316 [00:00<?, ? examples/s]

In [8]:
from itertools import chain

block_size = 512

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [9]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc = 4
)

Map (num_proc=4):   0%|          | 0/438316 [00:00<?, ? examples/s]

In [10]:
import s3fs

fs = s3fs.S3FileSystem(endpoint_url = 'http://minio:9000', anon = True)

In [11]:
lm_datasets.save_to_disk('s3://train/wiki-test', storage_options=fs.storage_options)

Saving the dataset (0/2 shards):   0%|          | 0/124569 [00:00<?, ? examples/s]